# k-Nearest Neighbors
Finds k nearest data points to a given data point and outputs majority vote value of output classes in case of classification, and average value of target values in case of regression. KNN was first added in MADlib 1.10 with multiple updates in subsequent releases.

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum Database 5.x on GCP (PM demo machine) - direct external IP access
#%sql postgresql://gpadmin@34.67.65.96:5432/madlib

# Greenplum Database 5.x on GCP (PM demo machine) - via tunnel
%sql postgresql://gpadmin@localhost:8000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

u'Connected: gpadmin@madlib'

In [3]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.17-dev, git revision: rel/v1.16-54-gec5614f, cmake configuration time: Wed Dec 18 17:08:05 UTC 2019, build type: release, build system: Linux-3.10.0-1062.4.3.el7.x86_64, C compiler: gcc 4.8.5, C++ compiler: g++ 4.8.5"


# 1.  Load data for classification

In [4]:
%%sql 
DROP TABLE IF EXISTS knn_train_data;

CREATE TABLE knn_train_data (
                    id integer, 
                    data integer[], 
                    label integer  -- Integer label means for classification
                    );

INSERT INTO knn_train_data VALUES
(1, '{1,1}', 1),
(2, '{2,2}', 1),
(3, '{3,3}', 1),
(4, '{4,4}', 1),
(5, '{4,5}', 1),
(6, '{20,50}', 0),
(7, '{10,31}', 0),
(8, '{81,13}', 0),
(9, '{1,111}', 0);

SELECT * FROM knn_train_data ORDER BY id;

Done.
Done.
9 rows affected.
9 rows affected.


id,data,label
1,"[1, 1]",1
2,"[2, 2]",1
3,"[3, 3]",1
4,"[4, 4]",1
5,"[4, 5]",1
6,"[20, 50]",0
7,"[10, 31]",0
8,"[81, 13]",0
9,"[1, 111]",0


# 2. Load data for regression

In [5]:
%%sql
DROP TABLE IF EXISTS knn_train_data_reg;

CREATE TABLE knn_train_data_reg (
                    id integer, 
                    data integer[], 
                    label float  -- Float label means for regression
                    );

INSERT INTO knn_train_data_reg VALUES
(1, '{1,1}', 1.0),
(2, '{2,2}', 1.0),
(3, '{3,3}', 1.0),
(4, '{4,4}', 1.0),
(5, '{4,5}', 1.0),
(6, '{20,50}', 0.0),
(7, '{10,31}', 0.0),
(8, '{81,13}', 0.0),
(9, '{1,111}', 0.0);

SELECT * FROM knn_train_data_reg ORDER BY id;

Done.
Done.
9 rows affected.
9 rows affected.


id,data,label
1,"[1, 1]",1.0
2,"[2, 2]",1.0
3,"[3, 3]",1.0
4,"[4, 4]",1.0
5,"[4, 5]",1.0
6,"[20, 50]",0.0
7,"[10, 31]",0.0
8,"[81, 13]",0.0
9,"[1, 111]",0.0


# 3. Load testing data

In [6]:
%%sql 
DROP TABLE IF EXISTS knn_test_data;

CREATE TABLE knn_test_data (
                    id integer, 
                    data integer[]
                    );

INSERT INTO knn_test_data VALUES
(1, '{2,1}'),
(2, '{2,6}'),
(3, '{15,40}'),
(4, '{12,1}'),
(5, '{2,90}'),
(6, '{50,45}');

SELECT * from knn_test_data ORDER BY id;

Done.
Done.
6 rows affected.
6 rows affected.


id,data
1,"[2, 1]"
2,"[2, 6]"
3,"[15, 40]"
4,"[12, 1]"
5,"[2, 90]"
6,"[50, 45]"


# 4. Run KNN for classification
Note that the nearest neighbors are sorted from closest to furthest from the corresponding test point.  Also note that the distance function is prepended with the schema where MADlib is installed (in this example 'madlib.squared_dist_norm2').

In [7]:
%%sql
DROP TABLE IF EXISTS knn_result_classification;

SELECT * FROM madlib.knn(
                'knn_train_data',      -- Table of training data
                'data',                -- Col name of training data
                'id',                  -- Col name of id in train data
                'label',               -- Training labels
                'knn_test_data',       -- Table of test data
                'data',                -- Col name of test data
                'id',                  -- Col name of id in test data
                'knn_result_classification',  -- Output table
                 3,                    -- Number of nearest neighbors
                 True,                 -- True to list nearest-neighbors by id
                 'madlib.squared_dist_norm2' -- Distance function
                );

SELECT * from knn_result_classification ORDER BY id;

Done.
1 rows affected.
6 rows affected.


id,data,prediction,k_nearest_neighbours,distance
1,"[2, 1]",1.0,"[1, 2, 3]","[1.0, 1.0, 5.0]"
2,"[2, 6]",1.0,"[5, 4, 3]","[5.0, 8.0, 10.0]"
3,"[15, 40]",0.0,"[7, 6, 5]","[106.0, 125.0, 1346.0]"
4,"[12, 1]",1.0,"[4, 5, 3]","[73.0, 80.0, 85.0]"
5,"[2, 90]",0.0,"[9, 6, 7]","[442.0, 1924.0, 3545.0]"
6,"[50, 45]",0.0,"[6, 7, 8]","[925.0, 1796.0, 1985.0]"


# 5. Run KNN for regression

In [8]:
%%sql
DROP TABLE IF EXISTS knn_result_regression;

SELECT * FROM madlib.knn(
                'knn_train_data_reg',  -- Table of training data
                'data',                -- Col name of training data
                'id',                  -- Col Name of id in train data
                'label',               -- Training labels
                'knn_test_data',       -- Table of test data
                'data',                -- Col name of test data
                'id',                  -- Col name of id in test data
                'knn_result_regression',  -- Output table
                 3,                    -- Number of nearest neighbors
                True,                  -- True to list nearest-neighbors by id
                'madlib.dist_norm2'    -- Distance function
                );

SELECT * FROM knn_result_regression ORDER BY id;

Done.
1 rows affected.
6 rows affected.


id,data,prediction,k_nearest_neighbours,distance
1,"[2, 1]",1.0,"[2, 1, 3]","[1.0, 1.0, 2.23606797749979]"
2,"[2, 6]",1.0,"[5, 4, 3]","[2.23606797749979, 2.82842712474619, 3.16227766016838]"
3,"[15, 40]",0.333333333333,"[7, 6, 5]","[10.295630140987, 11.1803398874989, 36.6878726556883]"
4,"[12, 1]",1.0,"[4, 5, 3]","[8.54400374531753, 8.94427190999916, 9.21954445729289]"
5,"[2, 90]",0.0,"[9, 6, 7]","[21.0237960416286, 43.8634243989226, 59.5399025864168]"
6,"[50, 45]",0.0,"[6, 7, 8]","[30.4138126514911, 42.3792402008342, 44.5533388198909]"


# 6. List nearest neighbors only
(without doing classification or regression).  Note that the nearest neighbors are sorted from closest to furthest from the corresponding test point.

In [9]:
%%sql
DROP TABLE IF EXISTS knn_result_list_neighbors;

SELECT * FROM madlib.knn(
                'knn_train_data_reg',  -- Table of training data
                'data',                -- Col name of training data
                'id',                  -- Col Name of id in train data
                NULL,                  -- NULL training labels means just list neighbors
                'knn_test_data',       -- Table of test data
                'data',                -- Col name of test data
                'id',                  -- Col name of id in test data
                'knn_result_list_neighbors', -- Output table
                3                      -- Number of nearest neighbors
                );

SELECT * FROM knn_result_list_neighbors ORDER BY id;

Done.
1 rows affected.
6 rows affected.


id,data,k_nearest_neighbours,distance
1,"[2, 1]","[2, 1, 3]","[1.0, 1.0, 5.0]"
2,"[2, 6]","[5, 4, 3]","[5.0, 8.0, 10.0]"
3,"[15, 40]","[7, 6, 5]","[106.0, 125.0, 1346.0]"
4,"[12, 1]","[4, 5, 3]","[73.0, 80.0, 85.0]"
5,"[2, 90]","[9, 6, 7]","[442.0, 1924.0, 3545.0]"
6,"[50, 45]","[6, 7, 8]","[925.0, 1796.0, 1985.0]"


# 7.  Weighted average
Run classification using weighted average

In [10]:
%%sql
DROP TABLE IF EXISTS knn_result_classification;

SELECT * FROM madlib.knn(
                'knn_train_data',      -- Table of training data
                'data',                -- Col name of training data
                'id',                  -- Col name of id in train data
                'label',               -- Training labels
                'knn_test_data',       -- Table of test data
                'data',                -- Col name of test data
                'id',                  -- Col name of id in test data
                'knn_result_classification',  -- Output table
                 3,                    -- Number of nearest neighbors
                 True,                 -- True to list nearest-neighbors by id
                 'madlib.squared_dist_norm2', -- Distance function
                 True                 -- For weighted average
                );

SELECT * FROM knn_result_classification ORDER BY id;

Done.
1 rows affected.
6 rows affected.


id,data,prediction,k_nearest_neighbours,distance
1,"[2, 1]",1,"[2, 1, 3]","[1.0, 1.0, 5.0]"
2,"[2, 6]",1,"[5, 4, 3]","[5.0, 8.0, 10.0]"
3,"[15, 40]",0,"[7, 6, 5]","[106.0, 125.0, 1346.0]"
4,"[12, 1]",1,"[4, 5, 3]","[73.0, 80.0, 85.0]"
5,"[2, 90]",0,"[9, 6, 7]","[442.0, 1924.0, 3545.0]"
6,"[50, 45]",0,"[6, 7, 8]","[925.0, 1796.0, 1985.0]"


# 8. Use kd-tree algorithm 
Here we build a kd-tree to depth 4 and search half (8) of the 16 leaf nodes (i.e., 2^4)

In [11]:
%%sql
DROP TABLE IF EXISTS knn_result_classification_kd;

SELECT madlib.knn(
                'knn_train_data',        -- Table of training data
                'data',                  -- Col name of training data
                'id',                    -- Col name of id in train data
                NULL,                    -- Training labels
                'knn_test_data',         -- Table of test data
                'data',                  -- Col name of test data
                'id',                    -- Col name of id in test data
                'knn_result_classification_kd',  -- Output table
                 3,                      -- Number of nearest neighbors
                 True,                   -- True to list nearest-neighbors by id
                 'madlib.squared_dist_norm2', -- Distance function
                 False,                  -- For weighted average
                 'kd_tree',              -- Use kd-tree
                 'depth=4, leaf_nodes=8' -- Kd-tree options
    
                 );
SELECT * FROM knn_result_classification_kd ORDER BY id;

Done.
1 rows affected.
6 rows affected.


id,data,k_nearest_neighbours,distance
1,"[2, 1]","[2, 1, 3]","[1.0, 1.0, 5.0]"
2,"[2, 6]","[5, 4, 3]","[5.0, 8.0, 10.0]"
3,"[15, 40]","[7, 6, 5]","[106.0, 125.0, 1346.0]"
4,"[12, 1]","[4, 5, 3]","[73.0, 80.0, 85.0]"
5,"[2, 90]","[9, 6, 7]","[442.0, 1924.0, 3545.0]"
6,"[50, 45]","[6, 7, 8]","[925.0, 1796.0, 1985.0]"


The result above is the same as brute force.  If we search just 1 leaf node, run-time will be faster but accuracy will be lower.  This shows up in this very small data set by not being able to find 3 nearest neighbors for all test points:

In [12]:
%%sql
DROP TABLE IF EXISTS knn_result_classification_kd;

SELECT madlib.knn(
                'knn_train_data',        -- Table of training data
                'data',                  -- Col name of training data
                'id',                    -- Col name of id in train data
                NULL,                    -- Training labels
                'knn_test_data',         -- Table of test data
                'data',                  -- Col name of test data
                'id',                    -- Col name of id in test data
                'knn_result_classification_kd',  -- Output table
                 3,                      -- Number of nearest neighbors
                 True,                   -- True to list nearest-neighbors by id
                 'madlib.squared_dist_norm2', -- Distance function
                 False,                  -- For weighted average
                 'kd_tree',              -- Use kd-tree
                 'depth=4, leaf_nodes=1' -- Kd-tree options
    
                 );
SELECT * FROM knn_result_classification_kd ORDER BY id;

Done.
1 rows affected.
5 rows affected.


id,data,k_nearest_neighbours,distance
1,"[2, 1]",[1],[1.0]
2,"[2, 6]","[3, 2]","[10.0, 16.0]"
3,"[15, 40]",[7],[106.0]
5,"[2, 90]","[3, 2]","[7570.0, 7744.0]"
6,"[50, 45]","[6, 8]","[925.0, 1985.0]"


# 9. Unsupervised nearest neighbors
Here the training set matches the test set so the nearest neighbor of each point is the point itself, with a zero distance.

In [13]:
%%sql
DROP TABLE IF EXISTS knn_result_classification_unsup;

SELECT * FROM madlib.knn(
                'knn_train_data',      -- Table of training data
                'data',                -- Col name of training data
                'id',                  -- Col name of id in train data
                 NULL,                 -- NULL training labels means just list neighbors
                'knn_train_data',      -- Table of test data (same as training data)
                'data',                -- Col name of test data
                'id',                  -- Col name of id in test data
                'knn_result_classification_unsup',  -- Output table
                 3,                    -- Number of nearest neighbors
                 True,                 -- True to list nearest-neighbors by id
                 'madlib.squared_dist_norm2' -- Distance function
                );

SELECT * from knn_result_classification_unsup ORDER BY id;

Done.
1 rows affected.
9 rows affected.


id,data,k_nearest_neighbours,distance
1,"[1, 1]","[1, 2, 3]","[0.0, 2.0, 8.0]"
2,"[2, 2]","[2, 3, 1]","[0.0, 2.0, 2.0]"
3,"[3, 3]","[3, 2, 4]","[0.0, 2.0, 2.0]"
4,"[4, 4]","[4, 5, 3]","[0.0, 1.0, 2.0]"
5,"[4, 5]","[5, 4, 3]","[0.0, 1.0, 5.0]"
6,"[20, 50]","[6, 7, 5]","[0.0, 461.0, 2281.0]"
7,"[10, 31]","[7, 6, 5]","[0.0, 461.0, 712.0]"
8,"[81, 13]","[8, 6, 7]","[0.0, 5090.0, 5365.0]"
9,"[1, 111]","[9, 6, 7]","[0.0, 4082.0, 6481.0]"


# 10. User defined distance function
The built-in distance functions are:
- dist_norm1: 1-norm/Manhattan
- dist_norm2: 2-norm/Euclidean
- squared_dist_norm2: squared Euclidean distance
- dist_angle: angle
- dist_tanimoto: tanimoto

You can create different distance functions via UDFs if desired.  For example, to create a Chebyshev distance function https://en.wikipedia.org/wiki/Chebyshev_distance :


In [14]:
%%sql
CREATE OR REPLACE FUNCTION chebychev_distance (x double precision[], y double precision[])
  RETURNS double precision
AS $$
    from scipy.spatial import distance
    return distance.chebyshev(x, y)
$$ LANGUAGE plpythonu;

Done.


[]

In [15]:
%%sql
DROP TABLE IF EXISTS knn_result_classification_udf;

SELECT * FROM madlib.knn(
                'knn_train_data',      -- Table of training data
                'data',                -- Col name of training data
                'id',                  -- Col name of id in train data
                'label',               -- Training labels
                'knn_test_data',       -- Table of test data
                'data',                -- Col name of test data
                'id',                  -- Col name of id in test data
                'knn_result_classification_udf',  -- Output table
                 3,                    -- Number of nearest neighbors
                 True,                 -- True to list nearest-neighbors by id
                 'chebychev_distance'  -- Distance function
                );

SELECT * from knn_result_classification_udf ORDER BY id;

Done.
1 rows affected.
6 rows affected.


id,data,prediction,k_nearest_neighbours,distance
1,"[2, 1]",1.0,"[1, 2, 3]","[1.0, 1.0, 2.0]"
2,"[2, 6]",1.0,"[5, 4, 3]","[2.0, 2.0, 3.0]"
3,"[15, 40]",0.0,"[7, 6, 5]","[9.0, 10.0, 35.0]"
4,"[12, 1]",1.0,"[5, 4, 3]","[8.0, 8.0, 9.0]"
5,"[2, 90]",0.0,"[9, 6, 7]","[21.0, 40.0, 59.0]"
6,"[50, 45]",0.0,"[6, 8, 7]","[30.0, 32.0, 40.0]"
